In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook as tqdm
import pickle

In [2]:
links = []
for page_num in tqdm(range(222)):
    response = requests.get(f"https://www.magyaridok.hu/page/{page_num + 1}/?after=2019-01-01&author&before=2019-12-31&category=0&s=+")
    soup = BeautifulSoup(response.content, 'html.parser')
    titles = soup.select("#left-area h2")
    links.extend([title.find("a").get("href") for title in titles])
with open('links.p', 'wb') as f:
    pickle.dump(links, f)

In [13]:
with open('links.p', 'rb') as f:
    links = pickle.load(f)

In [14]:
magyar_idok_dict = {}
for link in tqdm(links):
    response = requests.get(link)
    if response.status_code == 200:
        related_urls = []
        soup = BeautifulSoup(response.content, 'html.parser')
        datetime = soup.find("div", {"class": "en-article-dates"}).text.replace("\n", "")
        author = soup.find("div", {"class": "articleSourceAndAuthor"}).text.replace("\n", "").replace("\t", "")
        if "MTI" in author:
            related_urls.append("MTI")
        paragraphs = soup.find("div", {"class": "entry-content clearfix"}).find_all("p")
        for paragraph in paragraphs:
            if "MTI" in paragraph.text or "mti" in paragraph.text or "Mti" in paragraph.text:
                related_urls.append("MTI")
            if paragraph.find_all("a") is not None:
                for link in paragraph.find_all("a"):
                    if "http" in link.get('href'):
                        related_urls.append(link.get('href'))
                    else:
                        related_urls.append("https:" + link.get('href'))

        magyar_idok_dict[link] = {
        "related_links" : related_urls,
        "publish_date" : datetime
    }
    else:
        print(link)

In [10]:
import sys
sys.setrecursionlimit(50000)

In [15]:
with open('magyar_idok_dict.p', 'wb') as f:
    pickle.dump(magyar_idok_dict, f)

In [ ]:
f

In [22]:
for key in magyar_idok_dict:
    if len(magyar_idok_dict[key]["related_links"]) > 5:
        print(len(magyar_idok_dict[key]["related_links"]))

6
7
6
6
7
6
6
6
7
8
8


In [8]:
len(magyar_idok_dict)

4414